In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

df = pd.read_csv("train.csv")

df_apriori = df[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Survived']].copy()

df_apriori['Age'] = pd.cut(df_apriori['Age'].fillna(df_apriori['Age'].median()), bins=3, labels=["Young", "Middle", "Old"])
df_apriori['Fare'] = pd.cut(df_apriori['Fare'], bins=3, labels=["LowFare", "MidFare", "HighFare"])

df_apriori['Embarked'] = df_apriori['Embarked'].fillna('Missing')

for col in df_apriori.columns:
    df_apriori[col] = col + '=' + df_apriori[col].astype(str)

df_encoded = df_apriori.apply(lambda x: pd.Series(x)).stack().reset_index(drop=True)
df_encoded = pd.DataFrame({'Item': df_encoded})
df_encoded['Transaction'] = df_encoded.groupby(df_encoded.index // 6).ngroup()
basket = df_encoded.pivot_table(index='Transaction', columns='Item', aggfunc=lambda x: 1, fill_value=0)

frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)


print(rules.sort_values(by='lift', ascending=False).head(10))

                                 antecedents  \
1812  (Survived=1, Fare=LowFare, Embarked=S)   
1813                  (Sex=female, Pclass=2)   
1805    (Fare=LowFare, Sex=female, Pclass=2)   
1157                  (Sex=female, Pclass=2)   
1152                (Survived=1, Embarked=S)   
1820                (Survived=1, Embarked=S)   
1810  (Fare=LowFare, Sex=female, Embarked=S)   
1815                  (Survived=1, Pclass=2)   
1154                  (Survived=1, Pclass=2)   
1808    (Survived=1, Fare=LowFare, Pclass=2)   

                                 consequents  antecedent support  \
1812                  (Sex=female, Pclass=2)            0.237935   
1813  (Survived=1, Fare=LowFare, Embarked=S)            0.085297   
1805                (Survived=1, Embarked=S)            0.085297   
1157                (Survived=1, Embarked=S)            0.085297   
1152                  (Sex=female, Pclass=2)            0.243547   
1820    (Fare=LowFare, Sex=female, Pclass=2)            0.24354

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
